# **Setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Raster Images Generation**

**Connect to OpenAI:**

In [ ]:
!pip install openai

In [ ]:
!pip install openai pillow

In [ ]:
toekn = ''

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=token)

**Generate jpg images:**

In [ ]:
import os
import requests
from PIL import Image
from io import BytesIO

def get_url(prompt):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    return response.data[0].url

def save_image_as_jpg(prompt, directory, filename):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    # Generate the full path for the file
    file_path = os.path.join(directory, filename)

    # Download and save the image
    url = get_url(prompt)
    image_data = requests.get(url).content
    image = Image.open(BytesIO(image_data))
    rgb_image = image.convert('RGB')  # Convert to RGB if needed
    rgb_image.save(file_path, 'JPEG')


In [ ]:
objects = ['']

In [ ]:
import time
for i, obj in enumerate(objects, start=0):
    prompt = f"A black silhouette of a {obj} with well-defined main features that clearly outline the subject's overall shape. The silhouette shows high contrast between the sharp main contours and the bright white background.  The silhouette is very simple and lacks sharp edges and intricate details."
    save_image_as_jpg(prompt, '/content/drive/MyDrive/', f'image_{i}.jpg')
    print(f'file image_{i}.jpg saved {obj}')
    time.sleep(10)

file image_0.jpg saved panda


# **Images Description Generation**

**Conevert image path to url**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Loop through indices 1 to 2500 to get file IDs
def get_url_by_file_name(file_name):
    # Construct the file name based on the index
    # file_name = f"image_{ind}.jpg"

    # Query to find the file with the constructed name in the 'Image_dataset' folder
    query = f"'{image_dataset_folder_id}' in parents and title = '{file_name}' and trashed=false"
    file_list = drive.ListFile({'q': query}).GetList()

    # Check if the file was found and add to the dictionary
    if file_list:
        file_id = file_list[0]['id']
        url = f'https://drive.google.com/uc?export=download&id={file_id}'
        return url
    else:
        print(f"{file_name} not found.")
        return ''

**Query openAI GPT-4-turbo api**

In [ ]:
import requests

def get_description(url):
  payload =  {"model": "gpt-4-turbo",
    "messages": [
     {"role": "system",
      "content": [{"type": "text",
                   "text": "You are an image describer. Your goal is to give a title for the image."}],
     },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Describe in few words the scene in the image without any introduction like a title"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": url
            }
          }
        ]
      }
    ],
    "max_tokens": 500
  }

  headers = {"Authorization": f"Bearer {token}",
            "Content-Type": "application/json"}

  response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=payload)
  r = response.json()
  print(r)
  description = r["choices"][0]["message"]["content"].replace('"', '')
  return description

**Generate images descriptions**

In [ ]:
import os
import time

output_directory = ''

for i in range(2003, 2501):
    success = False
    retries = 3  # Number of times to retry if an error occurs
    while not success and retries > 0:
        try:
            url = get_url_by_file_name(f'image_{i}.jpg')
            print(url)
            description = get_description(url)
            text_filename = f"image_{i}.txt"
            text_filepath = os.path.join(output_directory, text_filename)
            with open(text_filepath, 'w') as f:
                f.write(description)
            print(f"{text_filename} saved")
            success = True
        except KeyError as e:
            print(f"Error occurred: {e}. Retrying...")
            retries -= 1
            if retries == 0:
                print(f"Failed to retrieve description for image_{i}. Skipping this file.")

https://drive.google.com/uc?export=download&id=12GCHQ89_saMmTNVhiQrbjq8XaQZcc3dF
{'id': 'chatcmpl-AIA5OTu7GZfttCIgiJA7OBjJV7xRI', 'object': 'chat.completion', 'created': 1728892038, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'A simple black silhouette of a cute, cartoon-style chick against a white background.', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 809, 'completion_tokens': 16, 'total_tokens': 825, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'system_fingerprint': 'fp_4dba7dd7b3'}
image_2003.txt saved
https://drive.google.com/uc?export=download&id=12K-eE4ZCh_bjMsQCztsBkBK_ZCACh-6d
{'id': 'chatcmpl-AIA5cDpYWy3YpbfeJHlz3zlXzR6en', 'object': 'chat.completion', 'created': 1728892052, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'A simple black silhouette of a small

# **Convert Raster Images to SVG**

In [ ]:
!apt-get install potrace
!pip install potracer[cli]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpotrace0
Suggested packages:
  netpbm
The following NEW packages will be installed:
  libpotrace0 potrace
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 89.0 kB of archives.
After this operation, 246 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libpotrace0 amd64 1.16-2 [17.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 potrace amd64 1.16-2 [71.8 kB]
Fetched 89.0 kB in 1s (67.2 kB/s)
Selecting previously unselected package libpotrace0:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libpotrace0_1.16-2_amd64.deb ...
Unpacking libpotrace0:amd64 (1.16-2) ...
Selecting previously unselected package potrace.
Preparing to unpack .../potrace_1.16-2_amd64.deb ...
Unpacking potrace (1.1

In [ ]:
import os
from PIL import Image
import subprocess

def process_image(image_path, output_folder):
    """
    Convert an image to grayscale, resize it to 500x500, save it as BMP, and then convert it to SVG using potrace.

    Parameters:
    - image_path: Path to the input image file.
    - output_folder: Path to the folder where the BMP and SVG files will be saved.
    """
    try:
        # Open the image file
        img = Image.open(image_path)
        base_name = os.path.splitext(os.path.basename(image_path))[0]

        # Resize the image to 500x500 pixels using LANCZOS resampling
        img_resized = img.resize((500, 500), Image.LANCZOS)

        # Convert the resized image to grayscale
        img_gray = img_resized.convert('L')

        # Save the grayscale image as a BMP file
        bmp_path = os.path.join(output_folder, f'{base_name}.bmp')
        img_gray.save(bmp_path)

        # Use potrace to convert the BMP file to an SVG file
        svg_path = os.path.join(output_folder, f'{base_name}.svg')
        subprocess.run(['potrace', bmp_path, '-s', '-o', svg_path], check=True)

        # Display a message
        print(f'SVG file saved to {svg_path}')
    except Exception as e:
        print(f'An error occurred while processing {image_path}: {e}')

def process_images_in_folder(folder_path, output_folder):
    """
    Process all images in the specified folder.

    Parameters:
    - folder_path: Path to the folder containing images.
    - output_folder: Path to the folder where the processed BMP and SVG files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # List of image formats
            image_path = os.path.join(folder_path, filename)
            process_image(image_path, output_folder)

**This is just code that removes the bmp files from the directory of the svg**

In [ ]:
import os

directory = '/content/drive/MyDrive/svg'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Check if the file ends with '.bmp'
    if filename.endswith('.bmp'):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        # Delete the file
        os.remove(file_path)
        print(f'Deleted {file_path}')


**SVG classification by number of tokens (at most 8k tokens is positive)**

In [ ]:
import tiktoken
import os
import shutil

# Define directories
source_dir = "/content/drive/MyDrive/svg"
dest_dir_pos = "/content/drive/MyDrive/SVG_dataset/pos"
dest_dir_neg = "/content/drive/MyDrive/SVG_dataset/neg"

# Create destination directories if they don't exist
os.makedirs(dest_dir_pos, exist_ok=True)
os.makedirs(dest_dir_neg, exist_ok=True)

# Function to count tokens in an SVG file
def count_tokens(text):
    # Initialize the tokenizer for GPT-3.5 model
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    return (len(tokens))

# Iterate over files in the source directory
for filename in os.listdir(source_dir):
    if filename.endswith(".svg"):
        svg_file_path = os.path.join(source_dir, filename)
        with open(svg_file_path, "r") as svg_file:
            svg_content = svg_file.read()

        # Count tokens in the SVG file
        token_count = count_tokens(svg_content)

        # Determine the destination directory based on token count
        if token_count <= 8000:
            dest_dir = dest_dir_pos
        else:
            dest_dir = dest_dir_neg

        # Move the SVG file
        shutil.move(svg_file_path, os.path.join(dest_dir, filename))


        print(f"Processed {filename} ({token_count} tokens)")


**Classify original images to pos and neg**

In [ ]:
import os
import shutil

# Define paths
image_dataset_path = "/content/drive/MyDrive/Text_dataset"
svg_dataset_path = "/content/drive/MyDrive/SVG_dataset"
classified_dataset_path = "/content/drive/MyDrive/Text_dataset_classified"

# Create classified directories
classified_pos_path = os.path.join(classified_dataset_path, "pos")
classified_neg_path = os.path.join(classified_dataset_path, "neg")
os.makedirs(classified_pos_path, exist_ok=True)
os.makedirs(classified_neg_path, exist_ok=True)

# Process SVG directories
for classification in ["pos", "neg"]:
    svg_class_path = os.path.join(svg_dataset_path, classification)
    classified_class_path = os.path.join(classified_dataset_path, classification)

    if not os.path.exists(svg_class_path):
        print(f"SVG folder {classification} does not exist. Skipping.")
        continue

    # Iterate over SVG files
    for svg_file in os.listdir(svg_class_path):
        if svg_file.endswith(".svg"):
            # Get the corresponding JPG file name
            jpg_file = svg_file.replace(".svg", ".txt")
            jpg_file_path = os.path.join(image_dataset_path, jpg_file)

            # Check if the JPG file exists
            if os.path.exists(jpg_file_path):
                # Move the JPG file to the corresponding classified folder
                shutil.copy(jpg_file_path, os.path.join(classified_class_path, jpg_file))
            else:
                print(f"JPG file {jpg_file} not found for SVG file {svg_file}. Skipping.")

print("Classification completed.")


Classification completed.


**Insert Text to Excel**

In [ ]:
import os
import pandas as pd

# Define paths
text_dataset_path = "/content/drive/MyDrive/Text_dataset"
output_excel_path = "/content/drive/MyDrive/Text_dataset.xlsx"

# Prepare data for the DataFrame
data = []

# Iterate over text files
for text_file in os.listdir(text_dataset_path):
    if text_file.endswith(".txt"):
        # Extract image name (without .txt extension)
        image_name = os.path.splitext(text_file)[0]

        # Read the content of the text file
        with open(os.path.join(text_dataset_path, text_file), "r", encoding="utf-8") as file:
            description = file.read().strip()

        # Append data to the list
        data.append({"image": image_name, "description": description})

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
df.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Excel file saved to {output_excel_path}")

Excel file saved to /content/drive/MyDrive/ShacharAndMaayan/Text_dataset.xlsx
